<a href="https://colab.research.google.com/github/pttxxxhit/Demographic-Data-Analyzer/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [4]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'
# de aca es cosecha
# 1) Leer archivos con las valoraciones y los libros
ratings = pd.read_csv('BX-Book-Ratings.csv', sep=';', encoding='latin-1')
books   = pd.read_csv('BX-Books.csv',        sep=';', encoding='latin-1', on_bad_lines='skip')

# 2) Quitar usuarios que votaron menos de 200 veces
user_counts   = ratings['User-ID'].value_counts()
active_users  = user_counts[user_counts >= 200].index
ratings       = ratings[ratings['User-ID'].isin(active_users)]

# 3) Quitar libros con menos de 100 valoraciones
book_counts   = ratings['ISBN'].value_counts()
popular_books = book_counts[book_counts >= 100].index
ratings       = ratings[ratings['ISBN'].isin(popular_books)]

--2025-10-19 03:06:35--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.4’

book-crossings.zip. 100%[===================>]  24.88M   150MB/s    in 0.2s    

2025-10-19 03:06:36 (150 MB/s) - ‘book-crossings.zip.4’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


/tmp/ipython-input-3316399263.py:11: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books   = pd.read_csv('BX-Books.csv',        sep=';', encoding='latin-1', on_bad_lines='skip')


In [6]:
# 1) Crear la matriz de votos: filas=libros, columnas=usuarios
matrix = ratings.pivot_table(
    index='ISBN',
    columns='User-ID',
    values='Book-Rating'
).fillna(0)

# 2) Guardar un diccionario ISBN → título
isbn_to_title = books.set_index('ISBN')['Book-Title'].to_dict()

# 3) Entrenar Vecinos Más Cercanos (KNN) con distancia coseno
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(matrix.values)

NearestNeighbors(algorithm='brute', metric='cosine')

In [7]:
# add your code here - consider creating a new cell for each section of code
def get_recommends(isbn, k=5):
    # 1) Sacar el vector de votos de ese libro
    book_vec = matrix.loc[isbn].values.reshape(1, -1)

    # 2) Pedir k+1 vecinos (incluye el mismo libro)
    distances, indices = model_knn.kneighbors(book_vec, n_neighbors=k+1)

    # 3) Preparar la lista de recomendaciones
    recs = []
    for i in range(1, k+1):
        idx      = indices[0][i]             # posición en matrix.index
        dist     = distances[0][i]           # qué tan lejos está
        rec_isbn = matrix.index[idx]         # ISBN recomendado
        title    = isbn_to_title.get(rec_isbn, 'Desconocido')
        recs.append([title, dist])

    # 4) Devolver [título original, lista de 5 recomendaciones]
    original = isbn_to_title.get(isbn, 'Desconocido')
    return [original, recs]

# Probar con un libro de ejemplo
print(get_recommends("0446605239"))

['The Notebook', [['A Walk to Remember', np.float64(0.6784089545022747)], ["The Pilot's Wife : A Novel", np.float64(0.8167747805926606)], ["Tuesdays with Morrie: An Old Man, a Young Man, and Life's Greatest Lesson", np.float64(0.8190190134424212)], ["Where the Heart Is (Oprah's Book Club (Paperback))", np.float64(0.8236682900571164)], ['The Red Tent (Bestselling Backlist)', np.float64(0.8253140278949362)]]]


In [11]:
# add your code here - consider creating a new cell for each section of code
def get_recommends(isbn, k=5):
    # 1) Sacar el vector de votos de ese libro
    book_vec = matrix.loc[isbn].values.reshape(1, -1)

    # 2) Pedir k+1 vecinos (incluye el mismo libro)
    distances, indices = model_knn.kneighbors(book_vec, n_neighbors=k+1)

    # 3) Preparar la lista de recomendaciones
    recs = []
    for i in range(1, k+1):
        idx      = indices[0][i]             # posición en matrix.index
        dist     = distances[0][i]           # qué tan lejos está
        rec_isbn = matrix.index[idx]         # ISBN recomendado
        title    = isbn_to_title.get(rec_isbn, 'Desconocido')
        recs.append([title, dist])

    # 4) Devolver [título original, lista de 5 recomendaciones]
    original = isbn_to_title.get(isbn, 'Desconocido')
    return [original, recs]

books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

KeyError: "Where the Heart Is (Oprah's Book Club (Paperback))"

In [12]:
# Buscar el ISBN para el libro "Where the Heart Is (Oprah's Book Club (Paperback))"
isbn_of_book = books[books['Book-Title'] == "Where the Heart Is (Oprah's Book Club (Paperback))"]['ISBN'].iloc[0]
print(f"El ISBN para 'Where the Heart Is (Oprah\'s Book Club (Paperback))' es: {isbn_of_book}")

El ISBN para 'Where the Heart Is (Oprah's Book Club (Paperback))' es: 0446672211
